# Fonctions Projet Wagon

## Fonction DF Stats from Nintendo

In [166]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
from pokemon_card_generator.data.pokemon_list import pokemon_list
import pickle

In [177]:
def stats_from_pkm_list(pklist: list,
                        df: bool = False,
                        parser: str = "html.parser",
                        x: int = 2,
                        verbose: bool = True):
    """ Function wich return Pandas dataframe or dictionary from pokemon list /
    :param pklist: Input string list of name,
    :param df: If this is True, function return directly a Pandas dataFrame
    :param parser: Differents parsers accepted "html.parser", "lxml" or "html5lib", for that think to install parser
    example: pip install html5lib
    :param x: Delay between each different request, warning, too short a delay can create a risk of ban
    :param verbose: Activate or not verbosity """

    uri = "https://www.pokemon.com/us/pokedex/"
    d = {}
    #
    for cpt,name in enumerate(pklist,1):
        url = f"{uri}{name.lower()}"
        if verbose:
            print(f"{cpt}/{len(pklist)}. Scrapping for {url} done !")
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        #
        span_title_list = soup.find_all("span", attrs={"attribute-title"})[0:5]
        span_value_list = soup.find_all("span", attrs={"attribute-value"})[0:5]
        d[name] = {}
        #
        for t, v in zip(span_title_list, span_value_list):
            d[name][t.text] = v.text
        # del d[name]['Gender']
        time.sleep(x)
    #
    if df:
        df = pd.concat(
            {k: pd.DataFrame.from_dict(v,'index')
             for k, v in d.items()},
            axis=1)
        df = df.T.reset_index().drop(columns=["level_1"])
        df = df.rename({"level_0":"Name"}, axis = 1)
        df = df.set_index("Name")
        return df
    else:
        return d

In [180]:
df_stats = stats_from_pkm_list(pokemon_list,df=True,x=1,parser='lxml')

1/898. Scrapping for https://www.pokemon.com/us/pokedex/bulbasaur done !
2/898. Scrapping for https://www.pokemon.com/us/pokedex/chikorita done !
3/898. Scrapping for https://www.pokemon.com/us/pokedex/treecko done !
4/898. Scrapping for https://www.pokemon.com/us/pokedex/turtwig done !
5/898. Scrapping for https://www.pokemon.com/us/pokedex/victini done !
6/898. Scrapping for https://www.pokemon.com/us/pokedex/chespin done !
7/898. Scrapping for https://www.pokemon.com/us/pokedex/rowlet done !
8/898. Scrapping for https://www.pokemon.com/us/pokedex/grookey done !
9/898. Scrapping for https://www.pokemon.com/us/pokedex/ivysaur done !
10/898. Scrapping for https://www.pokemon.com/us/pokedex/bayleef done !
11/898. Scrapping for https://www.pokemon.com/us/pokedex/grovyle done !
12/898. Scrapping for https://www.pokemon.com/us/pokedex/grotle done !
13/898. Scrapping for https://www.pokemon.com/us/pokedex/snivy done !
14/898. Scrapping for https://www.pokemon.com/us/pokedex/quilladin done !

114/898. Scrapping for https://www.pokemon.com/us/pokedex/ledian done !
115/898. Scrapping for https://www.pokemon.com/us/pokedex/silcoon done !
116/898. Scrapping for https://www.pokemon.com/us/pokedex/kricketot done !
117/898. Scrapping for https://www.pokemon.com/us/pokedex/stoutland done !
118/898. Scrapping for https://www.pokemon.com/us/pokedex/scatterbug done !
119/898. Scrapping for https://www.pokemon.com/us/pokedex/grubbin done !
120/898. Scrapping for https://www.pokemon.com/us/pokedex/blipbu done !
121/898. Scrapping for https://www.pokemon.com/us/pokedex/pidgey done !
122/898. Scrapping for https://www.pokemon.com/us/pokedex/spinarak done !
123/898. Scrapping for https://www.pokemon.com/us/pokedex/beautifly done !
124/898. Scrapping for https://www.pokemon.com/us/pokedex/kricketune done !
125/898. Scrapping for https://www.pokemon.com/us/pokedex/purrloin done !
126/898. Scrapping for https://www.pokemon.com/us/pokedex/spewpa done !
127/898. Scrapping for https://www.pokemo

227/898. Scrapping for https://www.pokemon.com/us/pokedex/ralts done !
228/898. Scrapping for https://www.pokemon.com/us/pokedex/combee done !
229/898. Scrapping for https://www.pokemon.com/us/pokedex/blitzle done !
230/898. Scrapping for https://www.pokemon.com/us/pokedex/meowstic done !
231/898. Scrapping for https://www.pokemon.com/us/pokedex/mudsdale done !
232/898. Scrapping for https://www.pokemon.com/us/pokedex/carko done !
233/898. Scrapping for https://www.pokemon.com/us/pokedex/nidorina done !
234/898. Scrapping for https://www.pokemon.com/us/pokedex/ampharos done !
235/898. Scrapping for https://www.pokemon.com/us/pokedex/kirlia done !
236/898. Scrapping for https://www.pokemon.com/us/pokedex/vespiquen done !
237/898. Scrapping for https://www.pokemon.com/us/pokedex/zebstrika done !
238/898. Scrapping for https://www.pokemon.com/us/pokedex/honedge done !
239/898. Scrapping for https://www.pokemon.com/us/pokedex/dewpider done !
240/898. Scrapping for https://www.pokemon.com/u

339/898. Scrapping for https://www.pokemon.com/us/pokedex/loudred done !
340/898. Scrapping for https://www.pokemon.com/us/pokedex/mismagius done !
341/898. Scrapping for https://www.pokemon.com/us/pokedex/palpitoad done !
342/898. Scrapping for https://www.pokemon.com/us/pokedex/clauncher done !
343/898. Scrapping for https://www.pokemon.com/us/pokedex/comfey done !
344/898. Scrapping for https://www.pokemon.com/us/pokedex/clobbopu done !
345/898. Scrapping for https://www.pokemon.com/us/pokedex/gloom done !
346/898. Scrapping for https://www.pokemon.com/us/pokedex/quagsire done !
347/898. Scrapping for https://www.pokemon.com/us/pokedex/exploud done !
348/898. Scrapping for https://www.pokemon.com/us/pokedex/honchkrow done !
349/898. Scrapping for https://www.pokemon.com/us/pokedex/seismitoad done !
350/898. Scrapping for https://www.pokemon.com/us/pokedex/clawitzer done !
351/898. Scrapping for https://www.pokemon.com/us/pokedex/oranguru done !
352/898. Scrapping for https://www.pok

451/898. Scrapping for https://www.pokemon.com/us/pokedex/medicham done !
452/898. Scrapping for https://www.pokemon.com/us/pokedex/gible done !
453/898. Scrapping for https://www.pokemon.com/us/pokedex/basculin done !
454/898. Scrapping for https://www.pokemon.com/us/pokedex/goodra done !
455/898. Scrapping for https://www.pokemon.com/us/pokedex/mimikyu done !
456/898. Scrapping for https://www.pokemon.com/us/pokedex/mr. rim done !
457/898. Scrapping for https://www.pokemon.com/us/pokedex/growlithe done !
458/898. Scrapping for https://www.pokemon.com/us/pokedex/snubbull done !
459/898. Scrapping for https://www.pokemon.com/us/pokedex/electrike done !
460/898. Scrapping for https://www.pokemon.com/us/pokedex/gabite done !
461/898. Scrapping for https://www.pokemon.com/us/pokedex/sandile done !
462/898. Scrapping for https://www.pokemon.com/us/pokedex/klefki done !
463/898. Scrapping for https://www.pokemon.com/us/pokedex/bruxish done !
464/898. Scrapping for https://www.pokemon.com/us

563/898. Scrapping for https://www.pokemon.com/us/pokedex/numel done !
564/898. Scrapping for https://www.pokemon.com/us/pokedex/lumineon done !
565/898. Scrapping for https://www.pokemon.com/us/pokedex/tirtouga done !
566/898. Scrapping for https://www.pokemon.com/us/pokedex/hoopa done !
567/898. Scrapping for https://www.pokemon.com/us/pokedex/lunala done !
568/898. Scrapping for https://www.pokemon.com/us/pokedex/dracozolt done !
569/898. Scrapping for https://www.pokemon.com/us/pokedex/tentacool done !
570/898. Scrapping for https://www.pokemon.com/us/pokedex/remoraid done !
571/898. Scrapping for https://www.pokemon.com/us/pokedex/camerupt done !
572/898. Scrapping for https://www.pokemon.com/us/pokedex/mantyke done !
573/898. Scrapping for https://www.pokemon.com/us/pokedex/carracosta done !
574/898. Scrapping for https://www.pokemon.com/us/pokedex/volcanion done !
575/898. Scrapping for https://www.pokemon.com/us/pokedex/nihilego done !
576/898. Scrapping for https://www.pokemon

675/898. Scrapping for https://www.pokemon.com/us/pokedex/dewgong done !
676/898. Scrapping for https://www.pokemon.com/us/pokedex/smoochum done !
677/898. Scrapping for https://www.pokemon.com/us/pokedex/solrock done !
678/898. Scrapping for https://www.pokemon.com/us/pokedex/mamoswine done !
679/898. Scrapping for https://www.pokemon.com/us/pokedex/ducklett done !
680/898. Scrapping for https://www.pokemon.com/us/pokedex/meltan done !
681/898. Scrapping for https://www.pokemon.com/us/pokedex/glastrier done !
682/898. Scrapping for https://www.pokemon.com/us/pokedex/grimer done !
683/898. Scrapping for https://www.pokemon.com/us/pokedex/elekid done !
684/898. Scrapping for https://www.pokemon.com/us/pokedex/barboach done !
685/898. Scrapping for https://www.pokemon.com/us/pokedex/porygon-z done !
686/898. Scrapping for https://www.pokemon.com/us/pokedex/swanna done !
687/898. Scrapping for https://www.pokemon.com/us/pokedex/melmetal done !
688/898. Scrapping for https://www.pokemon.co

788/898. Scrapping for https://www.pokemon.com/us/pokedex/glalie done !
789/898. Scrapping for https://www.pokemon.com/us/pokedex/eelektros done !
790/898. Scrapping for https://www.pokemon.com/us/pokedex/rhydon done !
791/898. Scrapping for https://www.pokemon.com/us/pokedex/spheal done !
792/898. Scrapping for https://www.pokemon.com/us/pokedex/elgye done !
793/898. Scrapping for https://www.pokemon.com/us/pokedex/chansey done !
794/898. Scrapping for https://www.pokemon.com/us/pokedex/sealeo done !
795/898. Scrapping for https://www.pokemon.com/us/pokedex/beheeye done !
796/898. Scrapping for https://www.pokemon.com/us/pokedex/tangela done !
797/898. Scrapping for https://www.pokemon.com/us/pokedex/walrein done !
798/898. Scrapping for https://www.pokemon.com/us/pokedex/litwic done !
799/898. Scrapping for https://www.pokemon.com/us/pokedex/kangaskhan done !
800/898. Scrapping for https://www.pokemon.com/us/pokedex/clamperl done !
801/898. Scrapping for https://www.pokemon.com/us/po

TypeError: 'Index' object is not callable

In [184]:
listpk = df_stats.index.to_list()

In [186]:
listk = [item for item in pokemon_list if item not in listpk]

In [188]:
listk

['Thwacke',
 'Rillaboo',
 'Raboo',
 'Cinderac',
 'Drizzil',
 'Inteleo',
 'Skwove',
 'Greeden',
 'Rookide',
 'Corvisquir',
 'Corviknigh',
 'Blipbu',
 'Dottle',
 'Orbeetl',
 'Nicki',
 'Thievu',
 'Gossifleu',
 'Eldegos',
 'Woolo',
 'Dubwoo',
 'Chewtl',
 'Pikachudagge',
 'Dredna',
 'Yampe',
 'Boltun',
 'Rolycol',
 'Nidoran♀',
 'Carko',
 'Coalossa',
 'Appli',
 'Nidoran♂',
 'Flappl',
 'Appletu',
 'Silicobr',
 'Sandacond',
 'Cramoran',
 'Arrokud',
 'Barraskewd',
 'Toxtricit',
 'Sizzliped',
 'Centiskorc',
 'Clobbopu',
 'Grapploc',
 'Siniste',
 'Polteageis',
 'Hatenn',
 'Hattre',
 'Hatteren',
 'Impidim',
 'Type: Null',
 'Morgre',
 'Grimmsnar',
 'Mime Jr.',
 'Obstagoo',
 'Perrserke',
 'Cursol',
 "Sirfetch'd",
 'Mr. Rim',
 'Runerigu',
 'Milcer',
 'Alcremi',
 'Falink',
 'Pincurchi',
 'Sno',
 'Tapu Koko',
 'Frosmot',
 'Tapu Lele',
 'Stonjourne',
 'Tapu Bulu',
 'Eiscu',
 'Tapu Fini',
 'Indeede',
 'Morpek',
 'Cufan',
 'Copperaja',
 'Duraludo',
 'Dreep',
 'Drakloa',
 'Dragapul',
 "Farfetch'd",
 'Vanil

In [187]:
df_stats_error = stats_from_pkm_list(listk,df=True,x=1,parser='lxml')

1/137. Scrapping for https://www.pokemon.com/us/pokedex/thwacke done !
2/137. Scrapping for https://www.pokemon.com/us/pokedex/rillaboo done !
3/137. Scrapping for https://www.pokemon.com/us/pokedex/raboo done !
4/137. Scrapping for https://www.pokemon.com/us/pokedex/cinderac done !
5/137. Scrapping for https://www.pokemon.com/us/pokedex/drizzil done !
6/137. Scrapping for https://www.pokemon.com/us/pokedex/inteleo done !
7/137. Scrapping for https://www.pokemon.com/us/pokedex/skwove done !
8/137. Scrapping for https://www.pokemon.com/us/pokedex/greeden done !
9/137. Scrapping for https://www.pokemon.com/us/pokedex/rookide done !
10/137. Scrapping for https://www.pokemon.com/us/pokedex/corvisquir done !
11/137. Scrapping for https://www.pokemon.com/us/pokedex/corviknigh done !
12/137. Scrapping for https://www.pokemon.com/us/pokedex/blipbu done !
13/137. Scrapping for https://www.pokemon.com/us/pokedex/dottle done !
14/137. Scrapping for https://www.pokemon.com/us/pokedex/orbeetl done 

KeyboardInterrupt: 

In [174]:
# Export pickle file
with open("df_stats_lxml.pkl", "wb") as file:
    pickle.dump(df_stats, file)

## Next